In [1]:
import os

import numpy as np
import pandas as pd
import utils

# Notebook 01: Compile TEOTIL data for vassdragsområder

TEOTIL3 results are available for 2013 to 2023. TEOTIL2 output covers 1990 to 2022, data for the first 6 years (1990 to 1995) are not reliable as the data in Nivabasen are patchy. Instead, we should use results taken from John Rune's old Access database for this period.

This notebook performs the following steps for each vassdragsområde:

 1. Build a TEOTIL2 series using John Rune's data for 1990 to 1995 and TEOTIL2 results from 1996 to 2022.
    
 2. Get TEOTIL3 data for 2013 to 2023.
    
 3. Bias correct the TEOTIL2 data using the period of overlap (2013 to 2022).

In [2]:
# Periods of interest
st_yr = 1990
teo2_st_yr = 1996
teo2_end_yr = 2022
teo3_st_yr = 2013
teo3_end_yr = 2023  # Assumed to be report year
bias_st_yr = 2013
bias_end_yr = 2022

nve_data_yr = teo3_end_yr + 1

# Use 'risk' loss for standard annual reports. It's usually a good idea to run the
# notebook with 'annual' losses too, as it's useful for other projects
agri_loss_model = "risk"

vassom_list = list(range(1, 248)) + list(range(301, 316))

par_dict = {"p": "FOSFOR", "n": "NITROGEN"}
col_order = [
    "Par",
    "Vassom",
    "År",
    "Akvakultur",
    "Jordbruk",
    "Avløp",
    "Industri",
    "Bebygd",
    "Bakgrunn",
]

In [3]:
rep_dir = f"../report_data/{teo3_end_yr}"
if not os.path.exists(rep_dir):
    os.makedirs(rep_dir)

## 1. Build TEOTIL2 series

### 1.1. Read John Rune's data

In [4]:
# Read John Rune's data
jrs_data_fold = r"/home/jovyan/shared/common/JES/teotil2_data/john_rune_data"
names_dict = {
    "VASSOMR": "Vassom",
    "Year": "År",
    "akva": "Akvakultur",
    "jordbr": "Jordbruk",
    "befolkn": "Avløp",
    "ind": "Industri",
    "natur": "Bakgrunn",
}
df_list = []
for par, fname in par_dict.items():
    jrs_xlsx = os.path.join(jrs_data_fold, f"t_{fname}.xlsx")
    jrs_df = (
        pd.read_excel(jrs_xlsx)
        .query("Year < @teo2_st_yr")
        .drop(columns=["ID1", "ID_Num", "Areal", "Q", "sum", "antrop"])
        .rename(columns=names_dict)
    )
    jrs_df["Par"] = par

    # John Rune doesn't explicitly include urban fluxes, but they are constant in TEOTIL2
    # Set to NaN here and then patch from later TEOTIL2 results below
    jrs_df["Bebygd"] = np.nan

    df_list.append(jrs_df)

jrs_df = pd.concat(df_list, axis="rows")
jrs_df = jrs_df[col_order]
jrs_df.head()

,Par,Vassom,År,Akvakultur,Jordbruk,Avløp,Industri,Bebygd,Bakgrunn
0,p,1,1990,0.000,12.169,5.477,15.800,NaN,4.388
1,p,2,1990,0.041,249.022,80.097,16.851,NaN,82.973
2,p,3,1990,0.000,14.837,6.760,18.021,NaN,1.536
3,p,4,1990,0.000,11.897,4.495,0.000,NaN,1.030
4,p,5,1990,0.000,6.579,4.860,0.000,NaN,0.800


In [5]:
# Get TEOTIL2 results
df_list = []
for par in par_dict.keys():
    teo2_df = utils.get_teotil2_results_for_vassoms(
        teo2_st_yr, teo2_end_yr, vassom_list
    )
    teo2_df = utils.aggregate_parameters(teo2_df, par, "teotil2")
    teo2_df["Par"] = par
    teo2_df["Vassom"] = teo2_df["regine"].astype(float).astype(int)
    del teo2_df["regine"]

    df_list.append(teo2_df)

teo2_df = pd.concat(df_list, axis="rows")
teo2_df = teo2_df[col_order]

# Merge with John Rune's data
teo2_df = (
    pd.concat([jrs_df, teo2_df], axis="rows")
    .sort_values(["Par", "Vassom", "År"])
    .reset_index(drop=True)
)

# Back-fill values for bebygd for early 1990s (they are constant in TEO2)
teo2_df['Bebygd'] = teo2_df['Bebygd'].bfill()

# Save
csv_path = f"../report_data/teotil2_vassom_np_{st_yr}-{teo2_end_yr}_patched_raw.csv"
teo2_df.to_csv(csv_path, index=False)

teo2_df.head()

,Par,Vassom,År,Akvakultur,Jordbruk,Avløp,Industri,Bebygd,Bakgrunn
0,n,1,1990,0.0,653.946,154.036,62.0,5.612849,512.925
1,n,1,1991,0.0,639.899,154.036,40.0,5.612849,474.674
2,n,1,1992,0.0,620.240,154.036,4.6,5.612849,474.118
3,n,1,1993,0.0,582.911,201.734,31.0,5.612849,449.825
4,n,1,1994,0.0,558.406,153.422,71.6,5.612849,554.214


## 2. Merge with TEOTIL3 and bias-correct

In [6]:
teo3_df = utils.get_teotil3_results_for_vassoms(
    teo3_st_yr,
    teo3_end_yr,
    vassom_list,
    agri_loss_model,
    nve_data_yr,
)

df_list = []
for par in par_dict.keys():
    # Get merged data for TEOTIL2
    teo2_par_df = teo2_df.query("Par == @par").drop(columns="Par").copy()

    # Standardise and aggregate output params for TEOTIL3
    teo3_par_df = utils.aggregate_parameters(teo3_df, par, "teotil3").copy()
    teo3_par_df["Vassom"] = teo3_par_df["regine"].astype(float).astype(int)
    del teo3_par_df["regine"]

    # Get TEOTIL2 results for period matching TEOTIL3 results
    # and calculate average for each vassom
    teo2_avg_par_df = (
        teo2_par_df.query("@bias_st_yr <= `År` <= @bias_end_yr")
        .drop(columns="År")
        .groupby("Vassom")
        .mean()
    )
    teo3_avg_par_df = (
        teo3_par_df.query("@bias_st_yr <= `År` <= @bias_end_yr")
        .drop(columns="År")
        .groupby("Vassom")
        .mean()
    )

    # Bias correct based on period for 2013 to 2022 for each vassom
    bias_df = (teo3_avg_par_df / teo2_avg_par_df).fillna(0).reset_index()
    bias_df.replace([np.inf, -np.inf], 1, inplace=True)
    bias_df = bias_df.set_index("Vassom").clip(lower=0, upper=10).reset_index()
    teo2_par_df = teo2_par_df.query("@st_yr <= `År` < @teo3_st_yr")
    teo2_par_df = pd.merge(
        teo2_par_df, bias_df, how="left", on="Vassom", suffixes=("", "_fac")
    )
    for src in bias_df.columns:
        if src != "Vassom":
            teo2_par_df[src] = teo2_par_df[src] * teo2_par_df[f"{src}_fac"]
            del teo2_par_df[f"{src}_fac"]

    # Merge bias-corrected TEOTIL2 data with TEOTIL3 output
    par_df = pd.concat([teo2_par_df, teo3_par_df], axis="rows")
    par_df["Par"] = par
    df_list.append(par_df)

df = (
    pd.concat(df_list, axis="rows")[col_order]
    .sort_values(["Par", "Vassom", "År"])
    .reset_index(drop=True)
)

# Calculate totals
df["Menneskeskapt"] = (
    df["Akvakultur"] + df["Jordbruk"] + df["Avløp"] + df["Industri"] + df["Bebygd"]
)
df["Totalt"] = df["Menneskeskapt"] + df["Bakgrunn"]

In [7]:
# Add TOC data from TEOTIL3
teo3_c_df = utils.aggregate_parameters(teo3_df, "c", "teotil3").copy()
teo3_c_df["Vassom"] = teo3_c_df["regine"].astype(float).astype(int)
del teo3_c_df["regine"]
teo3_c_df["Par"] = "c"
teo3_c_df = teo3_c_df[col_order]

# Calculate totals
teo3_c_df["Menneskeskapt"] = (
    teo3_c_df["Akvakultur"]
    + teo3_c_df["Jordbruk"]
    + teo3_c_df["Avløp"]
    + teo3_c_df["Industri"]
    + teo3_c_df["Bebygd"]
)
teo3_c_df["Totalt"] = teo3_c_df["Menneskeskapt"] + teo3_c_df["Bakgrunn"]

df = (
    pd.concat([df, teo3_c_df], axis="rows")
    .sort_values(["Par", "Vassom", "År"])
    .reset_index(drop=True)
)

# Save
csv_path = f"../report_data/{teo3_end_yr}/teotil2-3_vassom_npc_tonnes_{st_yr}-{teo3_end_yr}_agri-{agri_loss_model}-loss_bias-corrected.csv"
df.to_csv(csv_path, index=False)

df.head()

,Par,Vassom,År,Akvakultur,Jordbruk,Avløp,Industri,Bebygd,Bakgrunn,Menneskeskapt,Totalt
0,c,1,2013,0.0,6.994616,222.271973,444.0,63.091985,7297.738995,736.358574,8034.097569
1,c,1,2014,0.0,7.460279,247.727242,553.6,94.140966,10437.922634,902.928486,11340.851120
2,c,1,2015,0.0,6.678407,228.865176,543.6,86.443738,9660.041039,865.587321,10525.628360
3,c,1,2016,0.0,6.997184,349.106864,411.0,52.851431,6259.117020,819.955480,7079.072500
4,c,1,2017,0.0,8.022449,346.065040,877.6,49.156144,5885.895222,1280.843632,7166.738854
